In [48]:
import transformers
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
import os
import re
import random
import datasets
from datasets import load_dataset
from abc import ABC, abstractmethod
from typing import List, Optional, Tuple, Dict
from tqdm import tqdm
import pickle
from dotenv import load_dotenv
import openai
import json

from typing import List, Optional, Tuple, Dict
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset

import plotly.graph_objects as go
import plotly.express as px

import sys
sys.path.append('../')
from utils import untuple, eval_completions, levenshtein_distance
from act_add.contrast_dataset import ContrastDataset
from scripts.get_activations import gen_pile_data, compare_token_lists, slice_acts
from act_add.model_wrapper import ModelWrapper
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
with open('normal_generations.pkl', 'rb') as f:
    normal_generations = pickle.load(f)

with open('normal_generations_idxs.pkl', 'rb') as f:
    normal_generations_idxs = pickle.load(f)

with open('rej_generations4.pkl', 'rb') as f:
    rej_generations = pickle.load(f)

rej_generations

{3: ['\nMost Read\n\nMost Emailed\n\nMost Commented\n\nJoin the conversation:\n\nTo report inappropriate comments, abuse and/or repeat offenders please send an e-mail\nto social@post-trib.net with the subject "Post comment" and the following message in the body:\n"',
  '>Files</a><a class="SelectItem" href="javascript:void(0)" onclick="searchBox.OnSelectItem(3)"><span Class="">3.</Span><a id=3 href=""></a></div>\n</form>\n\n<script type=\'application\'>\n$(document).',
  'CTP                  = 0x84\n\tIPPROTO_TCP                   = 0x6\n\tIPPROTO_TP                    = 0x1d\n\tIPPROTO_TPXX                  =\\\n\tDLTXX_A                      = 00d5 | 00d1\n\tIPPROTO_IPV4                   =\\\n\t    IPV6           =',
  'SECONDARYCOLOR3UIPROC)load("glSecondaryColor3ui");\n\tglad_glSecondaryColor3uiv = (void*)_glfw.x11.secondaryColor3ui;\n#endif\n\tgladLoadGL();\n\n    if (ctxconfig.major!=',
  '488dp">488.00px</dimen>\n<dimen name="d_489dp">489.00px</dimen>\n<dimen name="w320dp" too

In [54]:
dists = levenshtein_distance(rej_generations[9], normal_generations)

len([x for x in dists if x > 0.9])

1

In [55]:
from transformers import AutoModelForCausalLM, AutoTokenizer

def calculate_perplexity(sentences, model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    total_loss = []
    total_tokens = 0

    for sentence in tqdm(sentences): 
        inputs = tokenizer(sentence, return_tensors="pt")
        input_ids = inputs['input_ids']
        attention_mask = inputs.attention_mask

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss
            num_tokens = input_ids.size(1)

        total_loss.append(loss.item())

    perplexity = np.exp(np.mean(total_loss))

    return perplexity

In [ ]:
def random_space_inserter(sentence, num_insertions): 
    for _ in range(num_insertions): 
        idx = random.randint(0, len(sentence))
        sentence = sentence[:idx] + ' ' + sentence[idx:]
    return sentence

In [61]:
sentences = normal_generations[:60]
sentences = [random_space_inserter(x, 10) for x in sentences]
model_name = "meta-llama/Llama-2-7b-hf"

perplexity = calculate_perplexity(sentences, model_name)
print(f"Perplexity: {perplexity}")


 95%|█████████▌| 57/60 [01:12<00:04,  1.43s/it]

In [60]:
sentences = rej_generations[5]
model_name = "meta-llama/Llama-2-7b-hf"

perplexity = calculate_perplexity(sentences, model_name)
print(f"Perplexity: {perplexity}")

# 12,11,9,9 ƒor 3,5,7,9

100%|██████████| 60/60 [01:16<00:00,  1.27s/it]


Perplexity: 11.489776090897074


In [20]:
rej_generations[3][:5]

['\nMost Read\n\nMost Emailed\n\nMost Commented\n\nJoin the conversation:\n\nTo report inappropriate comments, abuse and/or repeat offenders please send an e-mail\nto social@post-trib.net with the subject "Post comment" and the following message in the body:\n"',
 '>Files</a><a class="SelectItem" href="javascript:void(0)" onclick="searchBox.OnSelectItem(3)"><span Class="">3.</Span><a id=3 href=""></a></div>\n</form>\n\n<script type=\'application\'>\n$(document).',
 'CTP                  = 0x84\n\tIPPROTO_TCP                   = 0x6\n\tIPPROTO_TP                    = 0x1d\n\tIPPROTO_TPXX                  =\\\n\tDLTXX_A                      = 00d5 | 00d1\n\tIPPROTO_IPV4                   =\\\n\t    IPV6           =',
 'SECONDARYCOLOR3UIPROC)load("glSecondaryColor3ui");\n\tglad_glSecondaryColor3uiv = (void*)_glfw.x11.secondaryColor3ui;\n#endif\n\tgladLoadGL();\n\n    if (ctxconfig.major!=',
 '488dp">488.00px</dimen>\n<dimen name="d_489dp">489.00px</dimen>\n<dimen name="w320dp" tools:overr

In [22]:
normal_generations[:5]

['\nMost Read\n\nMost Emailed\n\nMost Commented\n\nJoin the conversation:\n\nTo report inappropriate comments, abuse and/or repeat offenders, please send an email to\nsocialmedia@post-gazette.com and include a link to the article and a copy of the comment. Your',
 '>Files</a><a class="SelectItem" href="javascript:void(0)" onclick="searchBox.OnSelectItem(3)"><span class="SelectionMark">&#160;</span>Functions</a><a class="SelectItem" href="javascript:void(0)" onclick="searchBox',
 'CTP                  = 0x84\n\tIPPROTO_TCP                   = 0x6\n\tIPPROTO_TP                    = 0x1d\n\tIPPROTO_UDP                   = 0x11\n\tIPV6_ADDRFORM                 = 0x16\n\tIPV6_ADDR_PREFERENCES         =',
 'SECONDARYCOLOR3UIPROC)load("glSecondaryColor3ui");\n\tglad_glSecondaryColor3uiv = (PFNGLSECONDARYCOLOR3UIVPROC)load("glSecondaryColor3uiv");\n\tglad_glSecondaryColor3us',
 '488dp">488.00px</dimen>\n<dimen name="d_489dp">489.00px</dimen>\n<dimen name="d_490dp">490.00px</dimen>\n<dimen name

In [32]:
len(rej_generations[3][0])

248

In [33]:
# sample random characters of length 248 

random_generations = []
for i in range(10):
    random_generations.append(''.join(random.choices('abcdefghijklmnopqrstuvwxyz', k=248)))

random_generations

['kewhjtsdprqnjgsfjvfiqpjzexksvztgxdczuaezazaoueyuafnhzqtwmjryurodvoiflaqwcsctkfbqpfiywpapuudnjqpcamhtlsascmbpuepnvyzfabfixnjngzrshftjogjcorpxcyeepgmpeduhgvxfooiyebchnpgtixzwovptzftnqurcynkdcfcgvdcfktltzofpiwixwtmfwmkmfeefrjfibqdgwbqqwwibdexruigoxcel',
 'hlgwsepeixyzceaxyweapqyahczosqnhymmnwjnyjufxtayfvoytbmhapgnciwzystgztixtfzldxddyzsctwyqnmsdronkapbbdzukporygwmzloweygqawulskdagyonsuqrprabqrghvxhfmaujntutyrdcbdvdijhgmxkuvgavmahzdoflorgwnkmbpwtbhaaaqcjpkziamvallntekwcdfoyrxbvjrhpcwodkzjlqauetahuujn',
 'chzhagwvjfizrkirtwdddoguhmiyooqecfuptvcwopvvmbknrchiyqauyxfwkttszffegggqvbjukpzynzrjxhxgyxsxefvtfxseyzjatdcixghkxawcsbvfjyjcisbltfyxezgtqbfdoeeyunzrgmkkhsfxfdhgqosxogvfemoxvwkshjwldezpuqygnysmhomfwvpkkgtulcymcbrjlippltugytbmydustgrnonzgywqhgpualkof',
 'cvwajypkdfpfwtfzbadlnoybvubvuhjrzwefuaptqwdcgumxidykadvwqufsdnpmhiklihcnoquyrfsrmknimzdynadqzzavaokywsvbmwdlydzsjrydcstsszuxehsdgsuwuoxsuhvlcwerzzpthvkhzrjswegjnwnwymrnhrtcmgmugiefcgivhbpcamwjzsbgwtxjswzeqwbcghdgsiodyzqnqtwvtetywrhjoxfhkzk

In [35]:
sentences = random_generations
model_name = "gpt2-xl"

perplexity = calculate_perplexity(sentences, model_name)
print(f"Perplexity: {perplexity}")

100%|██████████| 10/10 [00:05<00:00,  1.87it/s]


Perplexity: 313.8282355380864


In [36]:
from utils import sim_scores

sim_scores(rej_generations[3], normal_generations)

[0.8175355195999146,
 0.8833114504814148,
 0.872905969619751,
 0.779610812664032,
 0.7036529779434204,
 0.8883320093154907,
 0.9265872240066528,
 0.7103574275970459,
 0.6495053768157959,
 0.35981491208076477,
 0.9345995187759399,
 0.9550071358680725,
 0.879527747631073,
 0.9241190552711487,
 0.8780649304389954,
 0.9109675884246826,
 0.8953359127044678,
 0.7719061374664307,
 0.010516657494008541,
 0.9741053581237793,
 0.5982329845428467,
 0.5358380675315857,
 0.6615915298461914,
 0.8421670198440552,
 0.8800368309020996,
 0.5501348972320557,
 0.8350452780723572,
 0.8816800117492676,
 0.9332079887390137,
 0.7077783942222595,
 0.6410588026046753,
 0.9350363612174988,
 0.9135783314704895,
 0.9078866243362427,
 0.961298942565918,
 0.7989994883537292,
 -0.018304811790585518,
 0.8450167775154114,
 0.7194733023643494,
 0.8991549611091614,
 0.9600476026535034,
 0.9673956036567688,
 0.9614489674568176,
 0.7600520253181458,
 0.8342864513397217,
 0.5194719433784485,
 0.8574298024177551,
 0.89202487

In [47]:
sentence = "The quick brown fox jumps over the lazy dog"
# add spaces between each character
altered = 'The slow green fox shoots the lazy dog'
sim_scores([sentence], [altered])

[0.7570666074752808]

In [39]:
sentence, altered

('The quick brown fox jumps over the lazy dog',
 'T h e   q u i c k   b r o w n   f o x   j u m p s   o v e r   t h e   l a z y   d o g')